In [0]:
df = spark.table("workspace.default.finance_data")
#df.show(5)
print("Total_rows", df.count())
df.printSchema()
print("Total_rows", df.count())
df.dropna()
df.dropDuplicates()
print("Rows after cleaning:", df.count())
from pyspark.sql.functions import when 
df = df.withColumn("GenderGroup", when(df.age<25,"Young").when(df.age>=25, "Adult").when(df.age>=40, "middle").otherwise("seniorCitizen"))
#df.show(5)
from pyspark.sql.functions import when, col

df = df.withColumn("Investment_Avenues_Flag", when(col("Investment_Avenues") == "Yes", 1).otherwise(0)) \
       .withColumn("Stock_Market_Flag", when(col("Stock_Marktet") == "Yes", 1).otherwise(0))
#df.show(5)

numeric_cols = ["Mutual_Funds","Equity_Market","Debentures","Government_Bonds",
                "Fixed_Deposits","PPF","Gold"]

for c in numeric_cols:
    df = df.withColumn(f"{c}_Norm", (col(c) - 1) / 6.0)

df = df.withColumn("Total_Investment_Score",
    col("Mutual_Funds") + col("Equity_Market") + col("Debentures") +
    col("Government_Bonds") + col("Fixed_Deposits") + col("PPF") + col("Gold"))

df.select("age", "GenderGroup", "Investment_Avenues", "Investment_Avenues_Flag",
          "Mutual_Funds", "Mutual_Funds_Norm", "Stock_Marktet", "Stock_Market_Flag",
          "Total_Investment_Score")

#df.select("age", "GenderGroup", "Investment_Avenues", "Investment_Avenues_Flag",
#          "Mutual_Funds", "Mutual_Funds_Norm", "Stock_Marktet", "Stock_Market_Flag",
 #         "Total_Investment_Score").show(5)

from pyspark.sql.functions import avg, round

# Average normalized investment preferences by Age Group & Gender
agg_df = df.groupBy("GenderGroup", "gender").agg(
    round(avg("Mutual_Funds_Norm"), 2).alias("Avg_MutualFunds"),
    round(avg("Equity_Market_Norm"), 2).alias("Avg_Equity"),
    round(avg("Debentures_Norm"), 2).alias("Avg_Debentures"),
    round(avg("Government_Bonds_Norm"), 2).alias("Avg_Bonds"),
    round(avg("Fixed_Deposits_Norm"), 2).alias("Avg_FD"),
    round(avg("PPF_Norm"), 2).alias("Avg_PPF"),
    round(avg("Gold_Norm"), 2).alias("Avg_Gold"),
    round(avg("Total_Investment_Score"), 2).alias("Avg_TotalScore")
)

# Show aggregated results
#agg_df.show(truncate=False)


from pyspark.sql.window import Window

from pyspark.sql.functions import rank, col

wf = Window.partitionBy("GenderGroup").orderBy(col("Total_Investment_Score").desc())

rank = df.withColumn("Rank", rank().over(wf))

rank_df = rank.orderBy("GenderGroup")


#rank_df = rank.orderBy("GenderGroup").show(20, truncate=False)

from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

# Define window spec
windowSpec = Window.partitionBy("GenderGroup").orderBy(col("Total_Investment_Score").desc())

# Add ranking column (do NOT overwrite with show())
ranked_df = df.withColumn("Rank_in_AgeGroup", rank().over(windowSpec))

# Filter top 3 per Age_Group
top3_df = ranked_df.filter(col("Rank_in_AgeGroup") <= 3)

# Show result
#top3_df.orderBy("GenderGroup", "Rank_in_AgeGroup").show(truncate=False)

top3_df.orderBy("GenderGroup", "Rank_in_AgeGroup").show(3, truncate=False)



Total_rows 40
root
 |-- gender: string (nullable = true)
 |-- age: long (nullable = true)
 |-- Investment_Avenues: string (nullable = true)
 |-- Mutual_Funds: long (nullable = true)
 |-- Equity_Market: long (nullable = true)
 |-- Debentures: long (nullable = true)
 |-- Government_Bonds: long (nullable = true)
 |-- Fixed_Deposits: long (nullable = true)
 |-- PPF: long (nullable = true)
 |-- Gold: long (nullable = true)
 |-- Stock_Marktet: string (nullable = true)
 |-- Factor: string (nullable = true)
 |-- Objective: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Invest_Monitor: string (nullable = true)
 |-- Expect: string (nullable = true)
 |-- Avenue: string (nullable = true)
 |-- What are your savings objectives?: string (nullable = true)
 |-- Reason_Equity: string (nullable = true)
 |-- Reason_Mutual: string (nullable = true)
 |-- Reason_Bonds: string (nullable = true)
 |-- Reason_FD: string (nullable = true)
 |-- Source: 